# Desafío: Inferencia de Tópicos con algoritmo de Maximización de Esperanza (EM)

_17 de octubre de 2021_

**Francisca Pinto**

**Ejercicio 1**

In [ ]:
!pip install matplotlib
!pip install seaborn

#dataframe y arrays
import pandas as pd
import numpy as np

#estadística
import scipy.stats as stats

#gráficos

import matplotlib.pyplot as plt
import seaborn as sns

#separación entrenamiento validación
from sklearn.model_selection import train_test_split, GridSearchCV

#pipeline
from sklearn.pipeline import Pipeline

#clasificadores
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score

#preprocesamiento
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer

#algoritmo de maximización de espperanza EM
from sklearn.mixture import GaussianMixture

#trabajo con archivos múltiples en carpetas
import glob
import os

#tokenizador
from sklearn.feature_extraction.text import CountVectorizer

#Entrenamiento no supervisado - Inferencia de tópicos con Latent Dirichlet Allocation
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
sns.set(font_scale = 0.7) #escalar fuente para acomodar a gráfico
sns.set_style("darkgrid")
plt.rcParams["figure.figsize"] = (5,5) #escalar gráficos
plt.rcParams["figure.dpi"] = 150 #DPI gráficos (también modifica tamaño)

In [ ]:
#apertura de archivo.csv

folderpath = os.getcwd()
files = glob.glob(folderpath + "\\dump\\*.csv")

files_list = []

for i in files:

  tmp = pd.read_csv(i)
  files_list.append(tmp)

df = pd.concat(files_list, axis = 0)

df.drop(columns = "Unnamed: 0", inplace = True)
df.rename(columns = {"0" : "Artist",
                     "1" :"Genre",
                     "2" : "Song",
                     "3" : "Lyrics"},
          inplace = True)

df.reset_index(drop = True,
              inplace = True)

df.info(verbose = True, show_counts = True)

Comentarios:

1. Luego de ver los resultados del método <code>info</code> se observa que el DataFrame no tiene datos nulos.
2. Se continúa creando la instancia de <code>CountVectorizer</code> para la tokenización de los términos de las letras de cada canción y se crea un DataFrame con las 5000 apariciones más frecuentes.

**Ejercicio 2**

In [ ]:
count_vectorizer = CountVectorizer(stop_words = "english")

count_vectorizer_fit = count_vectorizer.fit_transform(list(df["Lyrics"]))

words = count_vectorizer.get_feature_names() #extrae términos

words_freq = count_vectorizer_fit.toarray().sum(axis = 0) #extrae frecuencias de cada término

In [ ]:
words_freq_df_data = {"Word" : words,
                      "Frecuency" : words_freq}

In [ ]:
words_freq_df = pd.DataFrame(data = words_freq_df_data)

In [ ]:
words_freq_df.sort_values(by = ["Frecuency"],
                          ascending = False,
                          inplace = True)

In [ ]:
words_freq_df.reset_index(drop = True,
                          inplace = True)

In [ ]:
words_freq_df_5000 = words_freq_df[: 5000]

In [ ]:
words_freq_df_5000 #se imprime para revisar formato

,Word,Frecuency
0,like,19629
1,don,17398
2,know,14962
3,got,14171
4,just,13978
...,...,...
4995,crawled,31
4996,absolute,31
4997,thursday,31
4998,tweedle,31


**Ejercicio 3**

1. Se requiere que la información de las letras de cada canción sea tokenizada, pero además es necesario que estos datos se relacionen con los artistas y letras, por lo que se creará un DataFrame con las letras tokenizadas que será unido al original (eliminándole la columna <code>Lyrics</code>). Este será el DataFrame que será utilizado para la inferencia de tópicos.

In [ ]:
all_words_df = pd.DataFrame(data = count_vectorizer_fit.toarray(),
                                 columns = words)

In [ ]:
df_tokenized = df.drop(columns = ["Lyrics", "Song"]).merge(all_words_df,
                        left_index = True,
                        right_index = True,
                        how = "right")

Comentarios:

1. Ahora se realizará la separación de conjuntos con <code>train_test_split</code>. Posteriormente, se crea la grilla para obtener los hiperparámetros con mejor comportamiento para realizar el modelo.

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(df_tokenized.drop(columns = "Genre"),
                                                   df_tokenized["Genre"],
                                                   test_size = 0.33,
                                                   random_state = 3748)

MemoryError: Unable to allocate 3.47 GiB for an array with shape (49019, 9489) and data type int64

In [ ]:
params = {"n_components" : [5, 10, 15],
          "gamma" : [0.7, 0.5]
          }

model1 = LatentDirichletAllocation()

grid = GridSearchCV(estimator = model1,
                    param_grid = params,
                    cv = 10,
                    )

grid.fit(X_train, y_train)
grid_yhat = grid.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
df_tokenized

,Artist,Genre,00,000,007,01,019,02,0281,03,...,ça,éclaboussant,écraser,électrique,éses,ölén,ömhetens,ødeleggelse,úgy,ﬁnally
0,Anthrax,metal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Anthrax,metal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Anthrax,metal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Anthrax,metal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Anthrax,metal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9484,Weezer,rock,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9485,Weezer,rock,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9486,Weezer,rock,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9487,Weezer,rock,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
